    The data in the boxscores and Anthropometrics files both go through an initial cleaning using MS excel, then uploaded into the mySQL server using MySQL workbench. For the Anthropometrics file, the data was based off of the NBA draft combine information for the years 2000 to 2023. For those who did not attend the combine, there wasn't information. Missing information for height and weight was collected from the internet and imputed using MS excel.

In [4]:
import numpy as np
import scipy
from scipy import stats
import sklearn
import pandas as pd
import seaborn as sns

In [5]:
box_scores = pd.read_csv('NBA-BoxScores-2023-2024.csv')
#sorting the box scores by game_ID, team_ID and minutes played by player
box_scores_sorted = box_scores.sort_values(['GAME_ID','TEAM_ID','MIN'])
box_scores_sorted = box_scores_sorted.drop(['NICKNAME'],axis=1)

In [6]:
#this cell transforms the players registerred with 2 positions to just their first position
anthropometrics = pd.read_csv('NBA_Anthropometric.csv')
anthropometrics['position'] = anthropometrics['position'].str.split('/').str[0]

Function to find the potential matchups for a given player
psuedocode:
box_scores and anthropometrics are the pandas dataframes.
-given a player in box scores 
-find their position(guard or forward classification) in anthropometrics
-in the opposition lineup find players with same position classification.
-since the players are sorted by minutes played for the respective game and team in box_scores the opponents of interest for a player are ranked as follows: the player whose rank in minutes played is equal to player of interest is ranked first opponent of interest, ranked second would be the one with one rank higher minutes and third ranked would be the one with one rank lower minutes played.
-Incase the player is the highest ranked by minutes consider the two players with ranks below equivalent opponent to be 2 and 3 in matchup ranks.
-and if lowest minutes played consider 2nd last and 3rd last for opponents

In [7]:
player_position = anthropometrics.loc[anthropometrics['player_name'] == 'Jonas Valanciunas', 'position'].values
anthropometrics[anthropometrics['player_name'] == 'Jonas Valanciunas']

,player_name,position,height,height_with_shoes,weight,wingspan,draft_year


In [72]:
def Opponents_of_interest(playerName, gameID, box_scores, anthropometrics):
    # Find the player's position in the anthropometrics DataFrame
    player_position = anthropometrics.loc[anthropometrics['player_name'] == playerName,
                                          'position'].values[0]

    # Find the game and team for the given player in the box_scores DataFrame
    team = box_scores[(box_scores['PLAYER_NAME'] == playerName) & (box_scores['GAME_ID'] == gameID)
                     ]['TEAM_ABBREVIATION'].values[0]
    #the minutes a player played in the given game
    playerMinutes = box_scores[(box_scores['PLAYER_NAME'] == playerName) & (box_scores['GAME_ID'] == gameID)
                     ]['MIN'].values[0]

    # Get the opposition lineup with the same position classification
    """If they have the same gameID in box_scores dataset and they are on oppo and their position 
    is in the list associated with the player of interest's in the PositionMatchup dictionary then 
    they qualify to be in the players matchups then they are ranked by minutes played"""
    
    #dictionary with the NBA positions and their corresponding likely matchups ranked in likelihood
    PositionMatchup = {'PG': ['PG','SG','SF'], 'SG':['SG','SF','PG'], 'SF':['SF','PF','SG'], 
                   'PF':['PF','C','SF'], 'C':['C','PF','SF']}
    #df of all the players of the opponent team
    oppositionMatchups = box_scores[(box_scores['GAME_ID'] == gameID) & (
        box_scores['TEAM_ABBREVIATION'] != team)]
    #the absolute difference between minutes played by players in the opponent team and the player of interest sorted from
    #least minute difference to most
    oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
    oppositionMatchups.sort_values(by='minutesDifference')
    #list of opposition players who meet the position matchup criteria ranked from least minutes difference to most with
    #player of interest
    opps = []
    for opp in oppositionMatchups['PLAYER_NAME']:
        player_position_data = anthropometrics.loc[anthropometrics['player_name'
                                                                  ] == opp, 'position'].values
        if player_position_data.size > 0 and player_position_data[0] in PositionMatchup[
            player_position]:
            opps.append((opp))
    anthropometric_mean = (
    anthropometrics[anthropometrics['player_name'].isin(opps[0:3])]
    .drop(['position', 'height_with_shoes', 'draft_year', 'player_name'], axis=1)
    .mean()
    )

    # Retrieve the player's points scored
    points_scored = box_scores.loc[
        (box_scores['PLAYER_NAME'] == playerName) & (box_scores['GAME_ID'] == gameID),
        'PTS'
    ]
    points_scored = points_scored.iloc[0] if not points_scored.empty else 0

    # Combine the results into a dictionary
    result = anthropometric_mean.to_dict()
    result['points_scored'] = points_scored  # Add the points scored

    return result
    #return anthropometrics[anthropometrics['player_name'].isin(opps[0:3])].drop(['position','height_with_shoes','draft_year'],axis=1).drop(['player_name'], axis=1).mean(), box_scores[(box_scores['PLAYER_NAME'] == playerName) & (box_scores['GAME_ID'] == gameID)]['PTS']

In [75]:
pointGuards = anthropometrics[anthropometrics['position'] == 'PG']['player_name']
results = []
for index, row in box_scores.iterrows():
    if row['PLAYER_NAME'] in pointGuards.values:
        result = Opponents_of_interest(row['PLAYER_NAME'], row['GAME_ID'], box_scores, anthropometrics)  # Call the function
        results.append(result)

results_df = pd.DataFrame(results)

C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is tr

C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is tr

C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is tr

C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is tr

C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is tr

...

C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is tr

C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is tr

C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is tr

C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is tr

C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)
C:\Users\guman\AppData\Local\Temp\ipykernel_11824\2930916971.py:26: SettingWithCopyWarning: 
A value is tr

In [76]:
print(results_df)

          height      weight    wingspan  points_scored
0     198.333333  100.153333  208.283333              0
1     198.333333  100.153333  208.283333              2
2     195.580000   94.316667  213.360000             14
3     195.580000   94.316667  213.360000             27
4     195.580000   94.316667  213.360000              1
...          ...         ...         ...            ...
1518  188.596667   86.826667  203.836667             16
1519  188.596667   86.826667  203.836667             31
1520  192.616667   88.360000  211.880000             31
1521  192.616667   88.360000  211.880000              0
1522  192.616667   88.360000  211.880000             18

[1523 rows x 4 columns]


In [66]:
opps = Opponents_of_interest('Anthony Davis', 22301230, box_scores, anthropometrics)
#opps2 = Opponents_of_interest('Austin Reaves', 22301230, box_scores, anthropometrics)[0:3]
print(opps)

6597    16
Name: PTS, dtype: int64


C:\Users\guman\AppData\Local\Temp\ipykernel_11824\1491362558.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppositionMatchups['minutesDifference'] = abs(oppositionMatchups['MIN'] - playerMinutes)


In [26]:
#function to retrieve opposition players anthropometrics and stats of interest
print(anthropometrics[anthropometrics['player_name'].isin(opps)].drop(['position','height_with_shoes','draft_year'],axis=1).drop(['player_name'], axis=1).mean())
print(anthropometrics[anthropometrics['player_name'].isin(opps2)].drop(['position','height_with_shoes','draft_year'],axis=1).drop(['player_name'], axis=1).mean())

average = oppsdf.drop(['player_name'], axis=1).mean()


height      197.906667
weight      109.443333
wingspan    212.936667
dtype: float64
height      194.733333
weight       96.296667
wingspan    209.340000
dtype: float64


In [39]:
# Assuming the function is named `get_primary_defender_avg` and is defined as:
# def get_primary_defender_avg(player_name, game_id):
#     # Logic to calculate average height and weight of primary defenders
#     return avg_height, avg_weight

# Step 1: Define a helper function for use with apply
def compute_defender_averages(row):
    height, weight, wingspan = Opponents_of_interest(row['PLAYER_NAME'], row['GAME_ID'], box_scores, anthropometrics)
    return pd.Series({'DEFENDER_AVG_HEIGHT': height, 'DEFENDER_AVG_WEIGHT': weight, 'DEFENDER_AVG_WINGSPAN': wingspan})

# Step 2: Apply the function to each row in the dataframe
box_scores[['DEFENDER_AVG_HEIGHT', 'DEFENDER_AVG_WEIGHT', 'DEFENDER_AVG_WINGSPAN']] = box_scores.apply(compute_defender_averages, axis=1)

# Step 3: Inspect the updated dataframe
print(box_scores)

IndexError: index 0 is out of bounds for axis 0 with size 0